In [1]:
import sys
sys.path.append("/home/tunghoang/hieunn/pyvene")

In [2]:
import torch
import pyvene as pv
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

from pyvene import embed_to_distrib, top_vals, format_token
from pyvene import (
    IntervenableModel,
    VanillaIntervention, Intervention,
    RepresentationConfig,
    IntervenableConfig,
    ConstantSourceIntervention,
    LocalistRepresentationIntervention
)

In [3]:
model_name = "alignment-handbook/zephyr-7b-sft-full" # your HF model name.
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.bfloat16, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [5]:
def zeroout_intervention_fn(b, s): 
    print(b.shape)
    b[:,0] = 0. # 3rd position
    return b

pv_model = pv.IntervenableModel({
    "component": "model.layers[15].mlp.output", # string access
    "intervention": zeroout_intervention_fn}, model=model)

# run the intervened forward pass
orig_outputs, intervened_outputs = pv_model(
    tokenizer("The capital of Spain is", return_tensors="pt").to('cuda'),
    output_original_output=True
)

print(intervened_outputs.logits - orig_outputs.logits)

torch.Size([1, 6, 4096])
tensor([[[ 0.0625,  0.0625, -0.0039,  ...,  0.0625,  0.0312,  0.0312],
         [ 0.0000,  0.0000,  0.0156,  ...,  0.0000, -0.0312, -0.0312],
         [ 0.0312,  0.0312,  0.0156,  ...,  0.0000, -0.0312,  0.0000],
         [ 0.0312,  0.0312, -0.0234,  ...,  0.0000,  0.0312,  0.0312],
         [ 0.0000,  0.0000, -0.0234,  ...,  0.0000,  0.0000,  0.0312],
         [ 0.0000, -0.0312,  0.0078,  ..., -0.0312, -0.0312, -0.0312]]],
       device='cuda:0')


In [6]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
context = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
print(tokenizer.decode(context[0]))

<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s> 
<|user|>
How many helicopters can a human eat in one sitting?</s> 
<|assistant|>



In [8]:
# outputs = model.generate(context.cuda(), max_new_tokens=128) 
outputs = model.generate(context.cuda(), max_new_tokens=128) 
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<|system|>
You are a friendly chatbot who always responds in the style of a pirate</s> 
<|user|>
How many helicopters can a human eat in one sitting?</s> 
<|assistant|>
I do not have the ability to eat or consume anything. However, I can tell you that the number of helicopters a human can eat in one sitting depends on the size and weight of the helicopters. It is highly unlikely that a human can eat a helicopter in one sitting, as they are typically too large and heavy to be consumed by a human.</s>


In [11]:
# res = pv_model(context.cuda())
res = pv_model(
    tokenizer("The capital of Spain is", return_tensors="pt").to('cuda'),
)
distrib = embed_to_distrib(pv_model, res.last_hidden_state, logits=False)
top_vals(tokenizer, distrib[0][-1], n=10)

torch.Size([1, 6, 4096])


NameError: name 'res' is not defined

In [8]:
pipe = pipeline(
    "text-generation",
    # model=model,
    model=pv_model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.1
)


print(pipe(messages, max_new_tokens=128)[0]['generated_text'][-1])

AttributeError: 'IntervenableModel' object has no attribute 'can_generate'

/home/tunghoang/anaconda3/envs/hnn-lm/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/tunghoang/anaconda3/envs/hnn-lm/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


{'role': 'assistant', 'content': 'I do not have information on how many helicopters a human can eat in one sitting. However, I suggest you consult with a medical professional or a nutritionist for accurate information regarding this matter.'}
